In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import urllib.request


In [2]:
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [4]:
dataset= pd.read_csv('C:\\Users\\AAA\\Desktop\\dev\\트위터 api\\house price.csv', encoding='cp949')
dataset= dataset.drop(['Datetime'], axis=1)
dataset.columns=['id','document','label']
dataset= dataset[:199]
dataset

,id,document,label
0,1.420000e+18,"RT @samsik_c: 작년에, 연말까지는 무조건 집값이 떨어진다고 정부가 나서서...",1.0
1,1.420000e+18,RT @wonnney: 대부분의 사람들이 모르는게.. 서울에 집을 많이 지으면 서울...,1.0
2,1.420000e+18,나도 다달이 집값으로 나가는 대출 이자 생각하면???\n(심지어 자가도 아닌\n절대...,0.0
3,1.420000e+18,"'미친 집값' 안떨어지는 이유, 중학교 사회교과서 답있다 [뉴스원샷] | 다음뉴스 ...",1.0
4,1.420000e+18,"""시체 위에 지어라"" 반발도···'미친 집값' 떨어지지 않는 이유 https://t...",1.0
...,...,...,...
194,1.420000e+18,"RT @goRaikkonen: ""해외 유학 대신"" 강남 부자들 우르르…13억 뚫은 ...",1.0
195,1.420000e+18,RT @lwhsongri5: 英 집값지수 13% 급등… ‘中억압 탈출’ 홍콩인들이 ...,1.0
196,1.420000e+18,RT @djWJrnwjWJrn: 집값은 아무래도 떨어지지않을건가봄 https://t...,1.0
197,1.420000e+18,英 집값지수 13% 급등… ‘中억압 탈출’ 홍콩인들이 런던 집 2000채 사들여 h...,1.0


In [5]:
dataset=dataset.astype({'label':'int64'})
dataset_train= dataset[:150]
dataset_test=dataset[150:199]

In [6]:
dataset_test.tail()

,id,document,label
194,1.420000e+18,"RT @goRaikkonen: ""해외 유학 대신"" 강남 부자들 우르르…13억 뚫은 ...",1
195,1.420000e+18,RT @lwhsongri5: 英 집값지수 13% 급등… ‘中억압 탈출’ 홍콩인들이 ...,1
196,1.420000e+18,RT @djWJrnwjWJrn: 집값은 아무래도 떨어지지않을건가봄 https://t...,1
197,1.420000e+18,英 집값지수 13% 급등… ‘中억압 탈출’ 홍콩인들이 런던 집 2000채 사들여 h...,1
198,1.420000e+18,'중국판 대치동'에 폭등..시진핑까지 나선 집값 잡기 | 다음 뉴스 https://...,1


In [6]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
GeForce GTX 1060 6GB


In [7]:
document_bert = ["[CLS] " + str(s) + " [SEP]" for s in dataset_train.document]
document_bert[:5]

['[CLS] RT @samsik_c: 작년에, 연말까지는 무조건 집값이 떨어진다고 정부가 나서서 국민들에게 집사지 마라던 대한민국 얘기 맞지요?\n\nhttps://t.co/lHflbVSRQW [SEP]',
 '[CLS] RT @wonnney: 대부분의 사람들이 모르는게.. 서울에 집을 많이 지으면 서울 집값은 더 많이 오르게 되어있다는 거...\n\n수요공급 법칙으로 되는게 아니라는걸 왜 모를까...\n\n집을 더 많이 지으면 그 지역 인구가 더 늘고, 그게 수요를 더 밀… [SEP]',
 '[CLS] 나도 다달이 집값으로 나가는 대출 이자 생각하면???\n(심지어 자가도 아닌\n절대 집에 있어\n뽕 뽑아야해 [SEP]',
 "[CLS] '미친 집값' 안떨어지는 이유, 중학교 사회교과서 답있다 [뉴스원샷] | 다음뉴스 https://t.co/a9tHxkZISm \n중학교 수준?? [SEP]",
 '[CLS] "시체 위에 지어라" 반발도···\'미친 집값\' 떨어지지 않는 이유 https://t.co/SdLOHJRFOI [SEP]']

In [8]:
#토크나이징

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in document_bert]
print(tokenized_texts[0])


['[CLS]', 'RT', '@', 'sam', '##sik', '_', 'c', ':', '작', '##년에', ',', '연', '##말', '##까지', '##는', '무', '##조', '##건', '집', '##값', '##이', '떨', '##어진', '##다고', '정', '##부가', '나', '##서', '##서', '국', '##민', '##들에게', '집', '##사', '##지', '마', '##라', '##던', '대한민국', '얘', '##기', '맞', '##지', '##요', '?', 'https', ':', '/', '/', 't', '.', 'co', '/', 'l', '##H', '##f', '##lb', '##V', '##SR', '##Q', '##W', '[SEP]']


In [9]:
#패딩
#token들의 max length보다 크게 MAX_LEN을 설정합니다.
#설정한 MAX_LEN 만큼 빈 공간을 0이 채웁니다
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
input_ids[0]

array([   101,  56898,    137,  21083,  41224,    168,    171,    131,
         9652,  27056,    117,   9568,  89523,  18382,  11018,   9294,
        20626,  71439,   9711, 118611,  10739,   9141,  46572,  85634,
         9670,  81896,   8982,  12424,  12424,   8909,  36553,  61688,
         9711,  12945,  12508,   9246,  17342,  23990,  26168,   9545,
        12310,   9256,  12508,  48549,    136,  14120,    131,    120,
          120,    188,    119,  11170,    120,    180,  12396,  10575,
        70832,  11779,  52238,  19282,  13034,    102,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [10]:
#어텐션 마스크
#학습속도를 높이기 위해 실 데이터가 있는 곳과 padding이 있는 곳을 attention에게 알려줍니다.

attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [11]:
train_inputs, validation_inputs, train_labels, validation_labels = \
train_test_split(input_ids, dataset_train['label'].values, random_state=42, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=42, 
                                                       test_size=0.1)

In [12]:
#파이토치 텐서로 변환

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [13]:
#배치 및 데이터로더 설정

BATCH_SIZE = 1

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

In [14]:
#테스트 셋에도 똑같이 적용

sentences = dataset_test['document']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
labels = dataset_test['label'].values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [15]:
#모델 학습

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1060 6GB


In [16]:
model = torch.load('C:\\Users\\AAA\\Desktop\\dev\\트위터 api\\model.pt')

model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [17]:
#학습스케쥴링
#transformers에서 제공하는 옵티마이저 중 AdamW를 사용합니다.
#총 훈련 스텝은 이터레이션 * 에폭수로 설정해둡니다.
#러닝 레잇 스케쥴러는 역시 transformers에서 제공하는 것을 사용합니다.

# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝
total_steps = len(train_dataloader) * epochs

# lr 조금씩 감소시키는 스케줄러
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [18]:
#학습
#accuracy와 시간함수

# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [19]:
#학습 실행부분
#데이터로더에서 배치만큼 가져온 후 forward, backward pass를 수행합니다.
#gradient update는 명시적으로 하지 않고 위에서 로드한 optimizer를 활용합니다.

# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 1.42
  Training epcoh took: 0:00:33

Running Validation...
  Accuracy: 0.80
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.42
  Training epcoh took: 0:00:32

Running Validation...
  Accuracy: 0.80
  Validation took: 0:00:01

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.36
  Training epcoh took: 0:00:32

Running Validation...
  Accuracy: 0.80
  Validation took: 0:00:01

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.27
  Training epcoh took: 0:00:32

Running Validation...
  Accuracy: 0.75
  Validation took: 0:00:01

Training complete!


In [20]:
torch.save(model, 'modeltrained.pt')

In [21]:
dataset_test.head()

,id,document,label
150,1.420000e+18,"“안타깝지만 집값 안 떨어집니다, 더 오를 겁니다” : 뉴스 : 동아닷컴 https...",1
151,1.420000e+18,"버블세븐 때처럼 '집값 거품론' 펴는 정부, 그런데도 시장은 정반대로 가고 있다 [...",1
152,1.420000e+18,https://t.co/X2BPCzsDxy\n공급폭탄이 정답입니다.\n오르는 집값 ...,0
153,1.420000e+18,RT @janghyeyeong: 대한민국 국회 기재위에서 부자감세 종부세 개악을 반...,0
154,1.420000e+18,"정부에서 아파트 가격 상한선을 결정하는 중국, 폭등 집값을 40%↓ https://...",0


In [22]:
t0 = time.time()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
lab=[]

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    lab.append(label_ids[0])
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

print(lab)


Accuracy: 0.65
Test took: 0:00:01
[1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [23]:
dataset_test['predicted']= lab
dataset_test.head(49)

c:\users\aaa\anaconda3\envs\pytorch\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,id,document,label,predicted
150,1.420000e+18,"“안타깝지만 집값 안 떨어집니다, 더 오를 겁니다” : 뉴스 : 동아닷컴 https...",1,1
151,1.420000e+18,"버블세븐 때처럼 '집값 거품론' 펴는 정부, 그런데도 시장은 정반대로 가고 있다 [...",1,0
152,1.420000e+18,https://t.co/X2BPCzsDxy\n공급폭탄이 정답입니다.\n오르는 집값 ...,0,1
153,1.420000e+18,RT @janghyeyeong: 대한민국 국회 기재위에서 부자감세 종부세 개악을 반...,0,0
154,1.420000e+18,"정부에서 아파트 가격 상한선을 결정하는 중국, 폭등 집값을 40%↓ https://...",0,1
155,1.420000e+18,RT @S710_VID: 마당 있는 큰 집 나올 때마다 부러워하며 가격부터 물어보는...,0,1
156,1.420000e+18,RT @djWJrnwjWJrn: 집값은 아무래도 떨어지지않을건가봄 https://t...,1,1
157,1.420000e+18,RT @hbeekay: @NYjjNY 맞는 말씀이시네요. ‘위기가 기회’라는 말을 ...,1,0
158,1.420000e+18,2Q 집값 서울은 부진전세 상승세 지속,1,0
159,1.420000e+18,＂해외 유학 대신＂ 강남 부자들 우르르…13억 뚫은 제주 집값 (출처 : 한국경제 ...,1,0


In [25]:
pos= 0
neg=0

for item in dataset_test['predicted']:
    if item==1:
        pos+=1
    else:
        neg+=1
        
        
print(pos, neg)

30 19


In [39]:
weight= pos/neg
weight

1.5789473684210527